In [1]:
from pprint import pprint
from fiftyone.types import COCODetectionDataset, YOLOv5Dataset
from fiftyone.utils.coco import COCODetectionDatasetImporter
import fiftyone.types
import fiftyone as fo
import fiftyone.utils.random as four
from fiftyone.utils.yolo import YOLOv5DatasetImporter
from pycocotools.coco import COCO
from pathlib import Path
import re
import json

In [31]:
# fo.delete_dataset('Russian-military')

In [4]:
ds2 = fo.Dataset('Russian-military')
data_dir2 = Path('Russian-military-annotated.v2-2022-04-25-rebalance-t-v-t-split.yolov7pytorch')

In [5]:
# The splits to load
splits = ['train', 'val', 'test']
for split in splits:
    ds2.add_dir(
        dataset_dir=data_dir2,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |█████████████████| 700/700 [689.1ms elapsed, 0s remaining, 1.0K samples/s]       
 100% |█████████████████| 193/193 [191.2ms elapsed, 0s remaining, 1.0K samples/s]     
 100% |█████████████████| 100/100 [95.9ms elapsed, 0s remaining, 1.0K samples/s]      


In [7]:
ds2

Name:        Russian-military
Media type:  image
Num samples: 993
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [6]:
ds2 = (ds2.set_field('tags', [])).clone()

In [7]:
ds2_class_mapping = {
    'bm-21': 'truck',
    'bmd-2': 'tank',
    'bmp-1': 'tank',
    'bmp-2': 'tank',
    'btr-70': 'tank',
    'btr-80': 'tank',
    'mt-lb': 'tank',
    't-64': 'tank',
    't-72': 'tank',
    't-80': 'tank',
}

In [8]:
view_ds2 = ds2.map_labels('ground_truth', ds2_class_mapping)

In [9]:
ds2 = view_ds2.clone()

In [10]:
four.random_split(ds2, {'train': 0.8, 'val': 0.2})

---

In [11]:
name = "MilitaryVehicles"
dataset_dir = Path('./Military and Civilian Vehicles Classification/yolov5_ds')

# The splits to load
splits = ['train',]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████| 6772/6772 [8.5s elapsed, 0s remaining, 824.5 samples/s]      


In [12]:
dataset = (dataset.set_field('tags', [])).clone()

In [13]:
labels_mapping = {'military tank': 'tank',
 'military aircraft': 'aircraft',
 'military truck': 'truck',
 'civilian aircraft': 'aircraft',
 'civilian car': 'car',
 'military helicopter': 'helicopter'
}

In [14]:
view_dataset = dataset.map_labels('ground_truth', labels_mapping)
dataset = view_dataset.clone()

In [15]:
four.random_split(dataset, {'train': 0.8, 'val': 0.2})

---

In [32]:
# fo.delete_dataset('CustomerDataset')

In [19]:
customer_ds = fo.Dataset.from_dir(
    dataset_dir='task_cv-nn-tanks-2023_03_12_19_21_38-coco 1.0',
    dataset_type=COCODetectionDataset,
    name='CustomerDataset',
)


 100% |███████████████████| 50/50 [142.1ms elapsed, 0s remaining, 351.9 samples/s] 


In [20]:
customer_ds = (customer_ds.set_field('tags', [])).clone()
customer_ds = customer_ds.map_labels('detections', labels_mapping).clone()
four.random_split(customer_ds, {'train': 0.8, 'val': 0.2})

In [21]:
export_dir = 'testset_yolov5'

classes = set(labels_mapping.values())
print(classes)

for split in ['train', 'val']:
    split_view = customer_ds.match_tags(split)
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        classes=classes,
    )

{'truck', 'aircraft', 'helicopter', 'car', 'tank'}
 100% |███████████████████| 40/40 [58.2ms elapsed, 0s remaining, 687.9 samples/s] 
Directory 'testset_yolov5' already exists; export will be merged with existing files
 100% |███████████████████| 10/10 [17.7ms elapsed, 0s remaining, 564.0 samples/s] 


In [22]:
name = "CustomerDsYolov5"

# The splits to load
splits = ['train', 'val']

# Load the dataset, using tags to mark the samples in each split
customer_ds_y5 = fo.Dataset(name)
for split in splits:
    customer_ds_y5.add_dir(
        dataset_dir='testset_yolov5',
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

 100% |███████████████████| 40/40 [59.2ms elapsed, 0s remaining, 675.6 samples/s]     
 100% |███████████████████| 10/10 [15.0ms elapsed, 0s remaining, 665.7 samples/s]    


In [23]:
customer_ds_y5

Name:        CustomerDsYolov5
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

---

In [24]:
_ = dataset.add_collection(ds2)
_ = dataset.add_collection(customer_ds_y5)

In [25]:
session = fo.launch_app(dataset)

In [29]:
export_dir = 'dataset_coco'

classes = set(labels_mapping.values())
print(classes)

for split in ['train', 'val']:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.COCODetectionDataset,
        classes=classes,
        labels_path=f'labels_{split}.json'
    )

{'truck', 'aircraft', 'helicopter', 'car', 'tank'}
 100% |███████████████| 6252/6252 [9.3s elapsed, 0s remaining, 485.2 samples/s]       
Directory 'dataset_coco' already exists; export will be merged with existing files
 100% |███████████████| 1563/1563 [2.7s elapsed, 0s remaining, 763.3 samples/s]      


In [30]:
export_dir = 'dataset_yolov5'

for split in ['train', 'val']:
    split_view = dataset.match_tags(split)
    split_view.export(
        export_dir=str(export_dir),
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        classes=classes,
    )

 100% |███████████████| 6252/6252 [6.0s elapsed, 0s remaining, 1.2K samples/s]        
Directory 'dataset_yolov5' already exists; export will be merged with existing files
 100% |███████████████| 1563/1563 [1.7s elapsed, 0s remaining, 893.9 samples/s]         
